In [48]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import scipy.stats as stats
from sklearn.metrics import r2_score

In [2]:
# sns.get_dataset_names()
df = sns.load_dataset("attention")
df = df.iloc[:,1:]
df.head()

,subject,attention,solutions,score
0,1,divided,1,2.0
1,2,divided,1,3.0
2,3,divided,1,3.0
3,4,divided,1,5.0
4,5,divided,1,4.0


In [3]:
df.dropna(inplace=True)

In [4]:
df.sample()

,subject,attention,solutions,score
29,10,divided,2,5.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   subject    60 non-null     int64  
 1   attention  60 non-null     object 
 2   solutions  60 non-null     int64  
 3   score      60 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.0+ KB


In [6]:
df.describe()

,subject,solutions,score
count,60.000000,60.000000,60.000000
mean,10.500000,2.000000,5.958333
std,5.814943,0.823387,1.621601
min,1.000000,1.000000,2.000000
25%,5.750000,1.000000,5.000000
50%,10.500000,2.000000,6.000000
75%,15.250000,3.000000,7.000000
max,20.000000,3.000000,9.000000


In [7]:
y = df["score"]
x = df[["subject","solutions"]]
x.sample(5)

,subject,solutions
58,19,3
37,18,2
30,11,2
55,16,3
42,3,3


<h1>Before box Cox Tranformation</h1>

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2)

In [10]:
model = LinearRegression()
model = model.fit(X_train,Y_train)

In [11]:
y_pred = model.predict(X_test)

In [12]:
y_pred

array([5.78662596, 6.52716839, 7.5602126 , 5.37588507, 6.40892928,
       4.5482113 , 6.88188572, 6.44003705, 5.10829907, 7.44197349,
       5.69949462, 5.02116774])

In [13]:
Y_test

42    6.0
33    8.0
57    6.0
8     2.0
32    5.0
1     3.0
36    7.0
17    7.0
21    4.0
56    6.0
26    4.5
5     5.0
Name: score, dtype: float64

In [14]:
model.intercept_

3.751645312235669

In [15]:
Y_test

42    6.0
33    8.0
57    6.0
8     2.0
32    5.0
1     3.0
36    7.0
17    7.0
21    4.0
56    6.0
26    4.5
5     5.0
Name: score, dtype: float64

In [34]:
r2_before_powerTransformation = r2_score(Y_test, y_pred)

In [35]:
r2_before_powerTransformation

0.2325409698768547

In [18]:
df.sample(5)

,subject,attention,solutions,score
30,11,focused,2,5.0
9,10,divided,1,6.0
32,13,focused,2,5.0
28,9,divided,2,3.0
39,20,focused,2,6.0


<h1>After Box Cox Transformation</h1>

In [19]:
from sklearn.preprocessing import PowerTransformer

In [20]:
tf = PowerTransformer(method="box-cox")

In [27]:
tf.lambdas_

array([0.66378666, 0.45144032])

In [21]:
X_train_transformed = tf.fit_transform(X_train)
X_test_transformed = tf.fit_transform(X_test)

In [22]:
model = model.fit(X_train_transformed,Y_train)
transformed_yPred = model.predict(X_test_transformed)

In [36]:
r2_after_powerTransformation = r2_score(transformed_yPred, y_pred)
r2_after_powerTransformation

0.9882009838601307

In [24]:
transformed_yPred

array([5.8571148 , 6.66509919, 7.55405973, 5.45254735, 6.56598689,
       4.48004037, 6.9491118 , 6.3522292 , 5.16775035, 7.46323235,
       5.90365108, 5.08917687])

In [59]:
score = pd.DataFrame({"before_transformation":np.array([r2_before_powerTransformation]),"after_transformation":np.array([r2_after_powerTransformation])})

In [60]:
score

,before_transformation,after_transformation
0,0.232541,0.988201


In [75]:
pd.DataFrame({
    "y_pred":y_pred,
    "transformed_yPred":transformed_yPred
})

,y_pred,transformed_yPred
0,5.786626,5.857115
1,6.527168,6.665099
2,7.560213,7.554060
3,5.375885,5.452547
4,6.408929,6.565987
5,4.548211,4.480040
6,6.881886,6.949112
7,6.440037,6.352229
8,5.108299,5.167750
9,7.441973,7.463232


<h1>yeo-johnson Tranformation</h1>

In [77]:
tf = PowerTransformer(method="box-cox")

In [78]:
X_train1_transformed = tf.fit_transform(X_train)
X_test1_transformed = tf.fit_transform(X_test)

In [79]:
model = model.fit(X_train1_transformed,Y_train)
transformed1_yPred = model.predict(X_test_transformed)